<a href="https://colab.research.google.com/github/sohomghosh/LIPI_ERAI_FinNLP_EMNLP-2022/blob/main/ERAI_SB_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Source: https://www.kaggle.com/code/sumantindurkhya/bert-for-regression/notebook

In [2]:
import numpy as np # linear algebra
import pandas as pd

import os


In [3]:
pd.set_option('precision', 20)

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 29.3 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 
     |████████████████████████████████| 120 kB 63.1 MB/s 


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertPreTrainedModel, BertModel
from transformers import AutoConfig, AutoTokenizer

from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [28]:
torch.__version__

'1.12.1+cu113'

In [29]:
import transformers
transformers.__version__

'4.22.1'

In [30]:
import sklearn
sklearn.__version__

'1.0.2'

In [7]:
def seed_everything(seed: int):
  import random, os
  import numpy as np
  import torch

  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(42)

In [ ]:
#unsupervised_test_gt = pd.read_json("ERAI_Dataset_unsupervised_test_GT.json").fillna(0)
#test_gt = pd.read_json("ERAI_Dataset_pairwise_test_GT.json")

In [9]:
train_df = pd.read_json("ERAI_Dataset_pairwise_train_with_translation.json").dropna()
train_data, validation = train_test_split(train_df, test_size=0.2, random_state=42)

train_data_formatted = pd.DataFrame({'index': list(train_data['index']) +list(train_data['index']), 'post_no':[1]*train_data.shape[0]+[2]*train_data.shape[0], 'post_rationale_1_2':list(train_data['post1'])+list(train_data['post2']), 'post_rationale_1_2_english': list(train_data['post1_en'])+list(train_data['post2_en']), 'MPP_1_2':list(train_data['MPP1'])+list(train_data['MPP2']), 'ML_1_2':list(train_data['ML1'])+list(train_data['ML2'])})
validation_formatted = pd.DataFrame({'index': list(validation['index']) +list(validation['index']), 'post_no':[1]*validation.shape[0]+[2]*validation.shape[0], 'post_rationale_1_2':list(validation['post1'])+list(validation['post2']), 'post_rationale_1_2_english': list(validation['post1_en'])+list(validation['post2_en']), 'MPP_1_2':list(validation['MPP1'])+list(validation['MPP2']), 'ML_1_2':list(validation['ML1'])+list(validation['ML2'])})

test_data_1 = pd.read_json("ERAI_Dataset_pairwise_test_with_translation.json")
test_data_1_formatted = pd.DataFrame({'index': list(test_data_1['index']) + list(test_data_1['index']), 'post_no':[1]*test_data_1.shape[0]+[2]*test_data_1.shape[0], 'post_rationale_1_2':list(test_data_1['post1'])+list(test_data_1['post2']), 'post_rationale_1_2_english': list(test_data_1['post1_en'])+list(test_data_1['post2_en'])})

unsupervised_test = pd.read_json("ERAI_Dataset_unsupervised_with_translation.json").rename(columns = {"post_rationale":"post_rationale_1_2", "post_rationale_en":"post_rationale_1_2_english"})


In [12]:
train_data.head(1)

,index,post1,post2,MPP1,MPP2,ML1,ML2,MPP_label,ML_label,post1_en,post2_en
5,10,水喔\n \n \n 〈中壽法說會〉去年隱含價值年增10.2% 每股EV為64.6元遠高於股...,中壽指出，6月並沒有特別實現股債利益，主要是股利已開始進帳，預估今年現金股利應可比去年增加1...,0.00302571860816900013,0.00469483568075100034,-0.03479576399394800068,-0.01721439749608700068,0,1,"Water\n \n \n ""Zhongshou Fa Said"" last year's ...","Zhongshou pointed out that in June, there was ..."


In [11]:
train_data_formatted.head()

,index,post_no,post_rationale_1_2,post_rationale_1_2_english,MPP_1_2,ML_1_2
0,10,1,水喔\n \n \n 〈中壽法說會〉去年隱含價值年增10.2% 每股EV為64.6元遠高於股...,"Water\n \n \n ""Zhongshou Fa Said"" last year's ...",0.00302571860816900013,-0.03479576399394800068
1,132,1,友達淨值約$18.16\n \n 10:59的買單拉抬到今日最高價$17.5....國際股市...,AUO net value is about $ 18.16\n \n 10:59 The ...,0.02179836512261500159,-0.04087193460490400054
2,278,1,"2330已經近二周沒甚麼消息,也許No news is good news,再過一日,202...",There is no news for 2330 for nearly two weeks...,0.07567567567567499986,-0.02522522522522500227
3,198,1,被這漲勢嚇到不知所措... \n 看來非核家園讓台汽電賺很大,I was scared by this rise ...\n It seems that ...,0.05525846702317200460,-0.01247771836007100128
4,342,1,漲了10%,10% increased,0.00649350649350600041,-0.03246753246753199956


In [13]:
test_data_1.head(1)

,index,post1,post2,post1_en,post2_en
0,400,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,神達:代子公司Silver Star Developments Ltd.公告董事會決議股利分...,I walked steadily these two days and felt good...,Shenda: Daizi Silver Star Developments LTD. An...


In [14]:
test_data_1_formatted.head()

,index,post_no,post_rationale_1_2,post_rationale_1_2_english
0,400,1,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,I walked steadily these two days and felt good...
1,402,1,話說統一獲利真的很穩定，每股盈餘年年都差不多2塊半左右，以前還有配股，今年公告僅配息2元，配...,It is said that the unified profit is really s...
2,404,1,統一這一波的漲勢真是猛。\n 有甚麼消息嗎？,The rise of this wave of unification is really...
3,406,1,統一這一波跌得真慘 \n 外資在股價跌到65的時候才降評，\n 是不是真的想要減碼啊？ \n...,The unified wave of this wave has fallen reall...
4,408,1,今天交完稅後，還有一點錢，\n 又進了一張群創，\n 進在11.3，\n 把套牢股票減少一點...,"After paying the tax today, there is still a l..."


# MPP (English, BERT)

In [15]:
MODEL_OUT_DIR = './output-finbert-regression'
## Model Configurations
MAX_LEN_TRAIN = 205
MAX_LEN_VALID = 205
MAX_LEN_TEST = 205
BATCH_SIZE = 64
LR = 1e-3
NUM_EPOCHS = 10
NUM_THREADS = 1  ## Number of threads for collecting dataset
MODEL_NAME = 'ProsusAI/finbert'

if not os.path.isdir(MODEL_OUT_DIR):
    os.makedirs(MODEL_OUT_DIR)

In [16]:
text = 'post_rationale_1_2_english'
label = 'MPP_1_2'
typ = 'MPP'

In [33]:
class Excerpt_Dataset(Dataset):

    def __init__(self, data, maxlen, tokenizer): 
        #Store the contents of the file in a pandas dataframe
        self.df = data.reset_index()
        #Initialize the tokenizer for the desired transformer model
        self.tokenizer = tokenizer
        #Maximum length of the tokens list to keep all the sequences of fixed size
        self.maxlen = maxlen

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):    
        #Select the sentence and label at the specified index in the data frame
        excerpt = self.df.loc[index, text]
        try:
            target = self.df.loc[index, label]
        except:
            target = 0.0
        #identifier = self.df.loc[index, 'id']
        #Preprocess the text to be suitable for the transformer
        tokens = self.tokenizer.tokenize(excerpt) 
        tokens = ['[CLS]'] + tokens + ['[SEP]'] 
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] 
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] 
        #Obtain the indices of the tokens in the BERT Vocabulary
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens) 
        input_ids = torch.tensor(input_ids) 
        #Obtain the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attention_mask = (input_ids != 0).long()
        
        target = torch.tensor(target, dtype=torch.float32)
        
        return input_ids, attention_mask, target

In [18]:
class BertRegresser(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        #The output layer that takes the [CLS] representation and gives an output
        self.cls_layer1 = nn.Linear(config.hidden_size,128)
        self.relu1 = nn.ReLU()
        self.ff1 = nn.Linear(128,128)
        self.tanh1 = nn.Tanh()
        self.ff2 = nn.Linear(128,1)

    def forward(self, input_ids, attention_mask):
        #Feed the input to Bert model to obtain contextualized representations
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #Obtain the representations of [CLS] heads
        logits = outputs.last_hidden_state[:,0,:]
        output = self.cls_layer1(logits)
        output = self.relu1(output)
        output = self.ff1(output)
        output = self.tanh1(output)
        output = self.ff2(output)
        return output

In [19]:
def train(model, criterion, optimizer, train_loader, val_loader, epochs, device):
    best_acc = 0
    for epoch in trange(epochs, desc="Epoch"):
        model.train()
        train_loss = 0
        for i, (input_ids, attention_mask, target) in enumerate(iterable=train_loader):
            optimizer.zero_grad()  
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            
            output = model(input_ids=input_ids, attention_mask=attention_mask)
            
            loss = criterion(output, target.type_as(output))
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        print(f"Training loss is {train_loss/len(train_loader)}")
        val_loss = evaluate(model=model, criterion=criterion, dataloader=val_loader, device=device)
        print("Epoch {} complete! Validation Loss : {}".format(epoch, val_loss))

In [20]:
def evaluate(model, criterion, dataloader, device):
    model.eval()
    mean_acc, mean_loss, count = 0, 0, 0

    with torch.no_grad():
        for input_ids, attention_mask, target in (dataloader):
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            output = model(input_ids, attention_mask)
            
            mean_loss += criterion(output, target.type_as(output)).item()
#             mean_err += get_rmse(output, target)
            count += 1
            
    return mean_loss/count

In [21]:
def get_rmse(output, target):
    err = torch.sqrt(metrics.mean_squared_error(target, output))
    return err

In [22]:
def predict(model, dataloader, device):
    predicted_label = []
    actual_label = []
    with torch.no_grad():
        for input_ids, attention_mask, target in (dataloader):
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            output = model(input_ids, attention_mask)
                        
            predicted_label += output
            actual_label += target
            
    return predicted_label

In [23]:
## Configuration loaded from AutoConfig 
config = AutoConfig.from_pretrained(MODEL_NAME)
## Tokenizer loaded from AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
## Creating the model from the desired transformer model
model = BertRegresser.from_pretrained(MODEL_NAME, config=config)
## GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## Putting model to device
model = model.to(device)
## Takes as the input the logits of the positive class and computes the binary cross-entropy 
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
## Optimizer
optimizer = optim.Adam(params=model.parameters(), lr=LR)

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at ProsusAI/finbert were not used when initializing BertRegresser: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertRegresser from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertRegresser from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertRegresser were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['ff2.weight', 'cls_layer1.bias', 'cls_layer1.weight', 'ff1.weight', 'ff2.bias', 'ff1.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
## Training Dataset
train_set = Excerpt_Dataset(data=train_data_formatted, maxlen=MAX_LEN_TRAIN, tokenizer=tokenizer)
valid_set = Excerpt_Dataset(data=validation_formatted, maxlen=MAX_LEN_VALID, tokenizer=tokenizer)
test_set = Excerpt_Dataset(data=test_data_1_formatted, maxlen=MAX_LEN_TEST, tokenizer=tokenizer)
unsupervised_test_set = Excerpt_Dataset(data=unsupervised_test, maxlen=MAX_LEN_TEST, tokenizer=tokenizer)


## Data Loaders
train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, num_workers=NUM_THREADS)
valid_loader = DataLoader(dataset=valid_set, batch_size=BATCH_SIZE, num_workers=NUM_THREADS)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, num_workers=NUM_THREADS)
unsupervised_test_loader = DataLoader(dataset=unsupervised_test_set, batch_size=BATCH_SIZE, num_workers=NUM_THREADS)

In [35]:
train(model=model, 
      criterion=criterion,
      optimizer=optimizer, 
      train_loader=train_loader,
      val_loader=valid_loader,
      epochs = 10,
     device = device)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([56])) that is different to the input size (torch.Size([56, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training loss is 0.00628459311556071


Token indices sequence length is longer than the specified maximum sequence length for this model (577 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch:  10%|█         | 1/10 [00:13<02:05, 13.94s/it]

Epoch 0 complete! Validation Loss : 0.0015154711436480284


Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors


...

Epoch 8 complete! Validation Loss : 0.000987076637102291


Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors


Training loss is 0.0006048042792826891


Token indices sequence length is longer than the specified maximum sequence length for this model (577 > 512). Running this sequence through the model will result in indexing errors
Epoch: 100%|██████████| 10/10 [01:57<00:00, 11.70s/it]

Epoch 9 complete! Validation Loss : 0.0009978441521525383


In [39]:
out1 = []
output = predict(model, train_loader, device)
for out in output:
    out1.append(out.cpu().detach().numpy()[0])
train_data_formatted[typ + '_1_2_predicted'] = out1
train_data_formatted_sorted = train_data_formatted.sort_values(['index', 'post_no']).copy()
train_data_formatted_sorted_grouped = train_data_formatted_sorted.groupby('index').agg({typ + '_1_2_predicted': lambda x:list(x)}).reset_index()
if typ=="MPP":
    train_data_formatted_sorted_grouped[typ + '_label_predicted'] = train_data_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 1 if x[0]>x[1] else 0)
else:
  train_data_formatted_sorted_grouped[typ + '_label_predicted'] = train_data_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 0 if x[0]>x[1] else 1)
train_data_with_prediction = train_data.merge(train_data_formatted_sorted_grouped, on = "index")
print(metrics.accuracy_score(train_data_with_prediction[typ + '_label'], train_data_with_prediction[typ + '_label_predicted']))

out2 = []
output = predict(model, valid_loader, device)
for out in output:
    out2.append(out.cpu().detach().numpy()[0])
validation_formatted[typ + '_1_2_predicted'] = out2
validation_formatted_sorted = validation_formatted.sort_values(['index', 'post_no']).copy()
validation_formatted_sorted_grouped = validation_formatted_sorted.groupby('index').agg({typ + '_1_2_predicted': lambda x:list(x)}).reset_index()
if typ=="MPP":
  validation_formatted_sorted_grouped[typ + '_label_predicted'] = validation_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 1 if x[0]>x[1] else 0)
else:
  validation_formatted_sorted_grouped[typ + '_label_predicted'] = validation_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 0 if x[0]>x[1] else 1)
validation_with_prediction = validation.merge(validation_formatted_sorted_grouped, on = "index")
print(metrics.accuracy_score(validation_with_prediction[typ + '_label'], validation_with_prediction[typ + '_label_predicted']))


out3 = []
output = predict(model, test_loader, device)
for out in output:
    out3.append(out.cpu().detach().numpy()[0])
test_data_1_formatted[typ + '_1_2_predicted'] = out3
test_data_1_formatted_sorted = test_data_1_formatted.sort_values(['index', 'post_no']).copy()
test_data_1_formatted_sorted_grouped = test_data_1_formatted_sorted.groupby('index').agg({typ + '_1_2_predicted': lambda x:list(x)}).reset_index()
if typ=="MPP":
  test_data_1_formatted_sorted_grouped[typ + '_label_predicted'] = test_data_1_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 1 if x[0]>x[1] else 0)
else:
  test_data_1_formatted_sorted_grouped[typ + '_label_predicted'] = test_data_1_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 0 if x[0]>x[1] else 1)
test_data_1_with_prediction = test_data_1.merge(test_data_1_formatted_sorted_grouped, on = "index")

out4 = []
output = predict(model, unsupervised_test_loader, device)
for out in output:
  out4.append(out.cpu().detach().numpy()[0])


unsupervised_test['MPP'] = out4
print(list(unsupervised_test.sort_values(typ, ascending = False if typ=="MPP" else True)['index']))

Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors


0.44871794871794873


Token indices sequence length is longer than the specified maximum sequence length for this model (577 > 512). Running this sequence through the model will result in indexing errors


0.525


Token indices sequence length is longer than the specified maximum sequence length for this model (734 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (842 > 512). Running this sequence through the model will result in indexing errors


[209, 186, 181, 163, 188, 43, 68, 199, 168, 167, 169, 185, 47, 178, 195, 165, 166, 204, 208, 32, 173, 130, 180, 202, 54, 205, 194, 174, 183, 156, 22, 64, 177, 196, 44, 175, 116, 157, 33, 140, 26, 187, 70, 41, 176, 148, 38, 203, 191, 138, 179, 39, 184, 119, 170, 114, 120, 206, 201, 200, 127, 76, 207, 71, 102, 152, 30, 51, 198, 197, 49, 46, 20, 172, 61, 145, 171, 81, 103, 193, 67, 117, 147, 105, 27, 79, 162, 50, 17, 143, 80, 113, 59, 66, 69, 60, 129, 111, 135, 72, 28, 94, 8, 29, 93, 150, 100, 164, 85, 158, 34, 37, 62, 14, 78, 101, 45, 133, 96, 139, 107, 104, 128, 132, 134, 23, 189, 31, 151, 2, 182, 40, 108, 7, 88, 42, 91, 92, 18, 159, 56, 112, 124, 82, 53, 25, 153, 55, 58, 65, 125, 89, 106, 192, 21, 13, 123, 73, 24, 16, 63, 110, 131, 74, 144, 19, 115, 57, 142, 83, 9, 95, 3, 87, 149, 1, 10, 84, 0, 136, 121, 52, 86, 160, 126, 99, 154, 15, 146, 118, 11, 137, 36, 190, 48, 5, 35, 77, 4, 122, 109, 161, 75, 141, 98, 97, 6, 12, 155, 90]


In [41]:
train_data_with_prediction.head(2)

,index,post1,post2,MPP1,MPP2,ML1,ML2,MPP_label,ML_label,post1_en,post2_en,MPP_1_2_predicted,MPP_label_predicted
0,10,水喔\n \n \n 〈中壽法說會〉去年隱含價值年增10.2% 每股EV為64.6元遠高於股...,中壽指出，6月並沒有特別實現股債利益，主要是股利已開始進帳，預估今年現金股利應可比去年增加1...,0.00302571860816900013,0.00469483568075100034,-0.03479576399394800068,-0.01721439749608700068,0,1,"Water\n \n \n ""Zhongshou Fa Said"" last year's ...","Zhongshou pointed out that in June, there was ...","[0.02207447588443756, 0.022074442356824875]",1
1,132,友達淨值約$18.16\n \n 10:59的買單拉抬到今日最高價$17.5....國際股市...,友達美股ADR昨天收盤才跌不到2.5趴\n 看來昨天友達跟群創跌過頭了…\n 新聞的消息應該...,0.02179836512261500159,0.01369863013698600050,-0.04087193460490400054,-0.09041095890410900948,1,0,AUO net value is about $ 18.16\n \n 10:59 The ...,AUO ADR only fell less than 2.5 lying yesterda...,"[0.022074498236179352, 0.02207452431321144]",0


In [42]:
validation_with_prediction.head(2)

,index,post1,post2,MPP1,MPP2,ML1,ML2,MPP_label,ML_label,post1_en,post2_en,MPP_1_2_predicted,MPP_label_predicted
0,284,"今天台股2330又將創新高?因為昨晚美股TSM收盤已來到140美元,相當於2330的793元...",在美國半導體鏈還沒壯大起來前. 敗登必需拉攏台積電.\n 台積電強大足以支撐整個台灣產鏈. ...,0.01435406698564500177,0.05641025641025600168,-0.03349282296650700486,0.00000000000000000000,0,1,"Today, Taiwan stocks 2330 will be innovative? ...","Before the US semiconductor chain grows, it mu...","[0.022074468433856964, 0.022074533626437187]",0
1,230,台積電ADR目前價位是114.60\n \n 換算下來是\n (114.60*27.9)/5...,600台🐔要搶嗎？\n 搶或不搶都是個難題,0.02113821138211300146,0.03691275167785200428,-0.00487804878048699996,-0.00167785234899300016,0,1,The current price of TSMC ADR is 114.60\n \n C...,600 units to grab?\n It is a difficult problem...,"[0.022074470296502113, 0.022074570879340172]",0


In [43]:
test_data_1_with_prediction.head(2)

,index,post1,post2,post1_en,post2_en,MPP_1_2_predicted,MPP_label_predicted
0,400,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,神達:代子公司Silver Star Developments Ltd.公告董事會決議股利分...,I walked steadily these two days and felt good...,Shenda: Daizi Silver Star Developments LTD. An...,"[0.022074487060308456, 0.022074449807405472]",1
1,402,話說統一獲利真的很穩定，每股盈餘年年都差不多2塊半左右，以前還有配股，今年公告僅配息2元，配...,向下探底，今天來到56.5元，以配息兩元，合理股價應該還有一段差距吧！讓我們再看下去,It is said that the unified profit is really s...,"Fix the bottom, come to 56.5 yuan today, with ...","[0.022074494510889053, 0.02207452617585659]",0


# For ML: Repeat the same